### Scrapy 
- 웹사이트에서 데이터를 수집하기위한 오픈 소스 프레임웍
    - https://scrapy.org/
    - framework란?
- 스크레피 구조
    - gmarket best 200 items 수집
- 프로젝트 실행 절차
    - scrapy 프로젝트 생성
    - 크롤링할 xpath 확인
    - items.py 코드 작성
    - spider.py 코드 작성
    - scrapy 프로젝트 실행

### 프로젝트 구조

- items.py
    - 모델(스키마)을 정의 : class 사용
- middlewares.py
    - user-agent와 같은 크롤링 설정값을 정의
- pipelines.py
    - 수집한 데이터를 출력하기 전에 어떤 코드(메신저로 데이터 전송, 이메일 전송, 디비에 저장)를 실행
- settings.py
    - robots.txt 를 따를지 안따를지, pipelines를 사용할지 안할지 등을 설정
- spiders
    - 어떤 웹 페이지를 크롤링할지 어떤 프로세스로 데이터를 수집할지 정의

### G마켓 베스트 상품 데이터 수집

#### 1. 스크레피 프로젝트 생성

In [1]:
!rm -rf crawler

In [2]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/notebooks/test/03_scrapy/gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [3]:
!ls

00_flake_pep8.ipynb	       04_scrapy_setting_naver_movie.ipynb
01_iterator_generator.ipynb    05_scrapy_argument_playnomore.ipynb
02_xpath.ipynb		       06_scrapy_selenium_naver_article.ipynb
03_scrapy_basic_gmarket.ipynb  gmarket


In [ ]:
# !sudo apt install tree

In [6]:
!tree gmarket

gmarket
├── gmarket
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

2 directories, 7 files


#### 2. 크롤링 데이터 xpath 확인
- 링크 데이터 수집
- 상세 페이지에서 상품 데이터 수집

In [9]:
import scrapy
import requests

from scrapy.http import TextResponse

In [10]:
# 1. 링크 데이터 수집 xpath

In [11]:
req = requests.get("http://corners.gmarket.co.kr/Bestsellers")
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [12]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/a/@href').extract()
len(links), links[0]

(200,
 'http://item.gmarket.co.kr/Item?goodscode=1919119193&ver=637406486679848203')

In [13]:
# 2. 상세 페이지에서 상품 데이터와 가격정보를 수집 xpath

In [14]:
links[2]

'http://item.gmarket.co.kr/Item?goodscode=1610385900&ver=637406486679848203'

In [15]:
req = requests.get(links[2])
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [16]:
title = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
s_price = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract()
try:
    o_price = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract()
except:
    o_price = s_price
title, o_price, s_price

('[맥콜] 맥콜 160ml 30캔x2 (총60캔) / 탄산음료', '14,900', '14,900')

#### 3. items.py 코드 작성

In [17]:
!tree gmarket/

gmarket/
├── gmarket
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

2 directories, 7 files


In [18]:
%%writefile gmarket/gmarket/items.py
import scrapy


class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    o_price = scrapy.Field()
    s_price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


#### 4. spider.py 코드 작성

In [19]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class Spider(scrapy.Spider):
    name = "GmarketBest"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
        item["s_price"] = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract()
        try:
            item["o_price"] = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract()
        except:
            item["o_price"] = item["s_price"]
        item["link"] = response.url
        yield item 

Writing gmarket/gmarket/spiders/spider.py


#### 5. 스크래피 실행
- scrapy.cfg 파일이 있는 디렉토리에서 아래의 명령 실행
    - $ scrapy crawl GmarketBest
    
- `-o gmarket.csv` 를 설정하면 csv 파일로 저장
    - $ scrapy crawl GmarketBest -o gmarket.csv

In [20]:
%%writefile run.sh
cd gmarket
rm gmarket.csv
scrapy crawl GmarketBest -o gmarket.csv

Writing run.sh


In [21]:
!ls

00_flake_pep8.ipynb		     05_scrapy_argument_playnomore.ipynb
01_iterator_generator.ipynb	     06_scrapy_selenium_naver_article.ipynb
02_xpath.ipynb			     gmarket
03_scrapy_basic_gmarket.ipynb	     run.sh
04_scrapy_setting_naver_movie.ipynb


In [22]:
!source run.sh

rm: cannot remove 'gmarket.csv': No such file or directory
2020-11-10 23:55:26 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: gmarket)
2020-11-10 23:55:26 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Oct 22 2020, 02:49:16) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.2.1, Platform Linux-5.4.0-1028-aws-x86_64-with-debian-buster-sid
2020-11-10 23:55:26 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-11-10 23:55:26 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'gmarket',
 'NEWSPIDER_MODULE': 'gmarket.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['gmarket.spiders']}
2020-11-10 23:55:26 [scrapy.extensions.telnet] INFO: Telnet Password: 41ce4f40736b5f56
2020-11-10 23:55:26 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.Telnet

2020-11-10 23:55:29 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1747419672&ver=637406493264503471>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1747419672&ver=637406493264503471',
 'o_price': '15,900',
 's_price': '12,900',
 'title': '[Aura] 아우라 건조기용 드라이시트 만다린 120매 +200ml 증정'}
2020-11-10 23:55:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1497316495&ver=637406493264503471> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:29 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1927088004&ver=637406493264503471>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1927088004&ver=637406493264503471',
 'o_price': '10,900',
 's_price': '10,900',
 'title': '[네파] 네파 스탠스 5족 번들 장목 남/여 택 1'}
2020-11-10 23:55:29 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1497316495&ver=637406493264503471>
{'link': 'http://i

2020-11-10 23:55:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1885549191&ver=637406493264543475> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:31 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1885549191&ver=637406493264543475>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1885549191&ver=637406493264543475',
 'o_price': '89,900',
 's_price': '79,900',
 'title': '[모르간] 20FW  MORGAN 울 헤링본 재킷'}
2020-11-10 23:55:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1899274105&ver=637406493264533474> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=173366583&ver=637406493264533474> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:32 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goo

2020-11-10 23:55:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1782687491&ver=637406493264533474> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1807976048&ver=637406493264533474> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:35 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=484828506&ver=637406493264533474>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=484828506&ver=637406493264533474',
 'o_price': '41,300',
 's_price': '28,900',
 'title': '[아침농산] 국산 농부의아침 쌀10kg  2020년산 햅쌀'}
2020-11-10 23:55:35 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1782687491&ver=637406493264533474>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1782687491&ver=637406493264533474',
 'o_price': '50,000',
 's_price': '49,000',
 'title': '[에쓰오일] 

2020-11-10 23:55:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1575961288&ver=637406493264533474> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:38 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1844379051&ver=637406493264533474>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1844379051&ver=637406493264533474',
 'o_price': '27,900',
 's_price': '27,400',
 'title': '[바이오더마] 바이오더마 클렌징워터 센시비오 펌프형 1+1  500ml+500ml'}
2020-11-10 23:55:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=731274728&ver=637406493264533474> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:38 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1575961288&ver=637406493264533474>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1575961288&ver=637406493264533474',
 'o_price': '80,000',
 's_price': '24,900',
 'ti

...

2020-11-10 23:55:50 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1551810301&ver=637406493264513472>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1551810301&ver=637406493264513472',
 'o_price': '80,000',
 's_price': '24,000',
 'title': '[바닐라코] 바닐라코 빅스 한정판 유니콘에디션 출시 프라이머1+1'}
2020-11-10 23:55:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1372301757&ver=637406493264513472> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:50 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1372301757&ver=637406493264513472>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1372301757&ver=637406493264513472',
 'o_price': '46,300',
 's_price': '13,900',
 'title': '20%쿠폰 남여경량패딩/조끼/기모후리스41종 5900원~'}
2020-11-10 23:55:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1438124143&ver=637406493264513472> (referer: http://co

2020-11-10 23:55:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1731920064&ver=637406493264513472>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1731920064&ver=637406493264513472',
 'o_price': '60,900',
 's_price': '60,900',
 'title': '한우 1등급  강병원 언양식 한판 구이 10판 (1.8kg)'}
2020-11-10 23:55:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=832114998&ver=637406493264513472> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=832114998&ver=637406493264513472>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=832114998&ver=637406493264513472',
 'o_price': '21,900',
 's_price': '6,900',
 'title': '로에랑스 퍼퓸 바디미스트/샤워코롱 250ml(+행사중)'}
2020-11-10 23:55:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1926145812&ver=637406493264513472> (referer: http://corners.gm

2020-11-10 23:55:57 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1870855588&ver=637406493264513472>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1870855588&ver=637406493264513472',
 'o_price': '25,350',
 's_price': '16,900',
 'title': '아삭 제철 사과 8kg (43-56과)'}
2020-11-10 23:55:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1494829360&ver=637406493264513472> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=679246929&ver=637406493264513472> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:55:57 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1494829360&ver=637406493264513472>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1494829360&ver=637406493264513472',
 'o_price': '25,150',
 's_price': '25,150',
 'title': '[피스넷] USB 충전식 손난로

2020-11-10 23:56:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1787515699&ver=637406493264503471> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:56:00 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1653023975&ver=637406493264503471>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1653023975&ver=637406493264503471',
 'o_price': '36,900',
 's_price': '36,900',
 'title': '[에이지투웨니스] 핑크다이아팩트 기획세트 21호'}
2020-11-10 23:56:00 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1787515699&ver=637406493264503471>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1787515699&ver=637406493264503471',
 'o_price': '121,900',
 's_price': '106,070',
 'title': '[닥터마틴] 닥터마틴  패트리샤 로퍼 BLACK'}
2020-11-10 23:56:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1507570213&ver=637406493264503471> (referer: http://corners.gmarket

2020-11-10 23:56:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1107162855&ver=637406493264503471> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-11-10 23:56:02 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1807812382&ver=637406493264503471>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1807812382&ver=637406493264503471',
 'o_price': '62,400',
 's_price': '31,200',
 'title': '[브라운물티슈] 브라운 아기물티슈 네이처 마일드 70매 캡형 10+10팩'}
2020-11-10 23:56:02 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1107162855&ver=637406493264503471>
{'link': 'http://item.gmarket.co.kr/Item?goodscode=1107162855&ver=637406493264503471',
 'o_price': '23,000',
 's_price': '6,900',
 'title': '[프롬산타] 3+1 양말/롱삭스/수면 남자여자학생스니커즈덧신겨울'}
2020-11-10 23:56:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1845754094&ver=637406493264503471> (referer: ht

In [23]:
import pandas as pd

In [24]:
df = pd.read_csv("gmarket/gmarket.csv")
df

,link,o_price,s_price,title
0,http://item.gmarket.co.kr/Item?goodscode=18958...,"25,900","25,900",[유한킴벌리] 크리넥스 비말차단스타일마스크흰색 성인용L45p(1BOX)
1,http://item.gmarket.co.kr/Item?goodscode=17489...,"16,900","16,740",알록달록 삶은 찰옥수수 30개 /냉동/대용량
2,http://item.gmarket.co.kr/Item?goodscode=15773...,"65,900","65,900",[종근당] 종근당건강 락토핏 생유산균 골드 5통 (250일분)
3,http://item.gmarket.co.kr/Item?goodscode=19199...,"45,000","45,000",강다니엘 1ST 아트북 - NEVER STANDING STILL
4,http://item.gmarket.co.kr/Item?goodscode=19135...,"17,500","17,500",국산 참맑은KF-AD비말차단마스크200매
...,...,...,...,...
195,http://item.gmarket.co.kr/Item?goodscode=97466...,"26,300","7,900",1+1 레깅스 기모/융털/밴딩팬츠/치마/빅사이즈~4XL
196,http://item.gmarket.co.kr/Item?goodscode=15089...,"47,000","47,000",[인가드] 인가드 국산 KF94 황사 미세먼지 마스크 대형 100매
197,http://item.gmarket.co.kr/Item?goodscode=19134...,"14,900","5,900",3초 바디 샤워가운 1+1 외 리빙용품 150종 +사은품
198,http://item.gmarket.co.kr/Item?goodscode=18401...,"50,000","50,000",[한국투자증권] (한국투자증권) 온라인 금융상품권 5만원권
